#### Import Libraries

In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

#### Load  Dataset

In [2]:
df=pd.read_csv('BlackFriday.csv')

In [3]:
pd.set_option('display.max_columns',None)
df

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969
...,...,...,...,...,...,...,...,...,...,...,...,...
550063,1006033,P00372445,M,51-55,13,B,1,1,20,NaN,NaN,368
550064,1006035,P00375436,F,26-35,1,C,3,0,20,NaN,NaN,371
550065,1006036,P00375436,F,26-35,15,B,4+,1,20,NaN,NaN,137
550066,1006038,P00375436,F,55+,1,C,2,0,20,NaN,NaN,365


#### Shape of Dataset

In [4]:
df.shape

(550068, 12)

#### Data Preprocessing

In [7]:
print(df['Age'].value_counts())
print(df['City_Category'].value_counts())
print(df['Stay_In_Current_City_Years'].value_counts())
print(df['Gender'].value_counts())
print(df['Occupation'].value_counts())
print(df['Marital_Status'].value_counts())
print(df['Product_Category_1'].value_counts())
print(df['Product_Category_2'].value_counts())
print(df['Product_Category_3'].value_counts())


26-35    219587
36-45    110013
18-25     99660
46-50     45701
51-55     38501
55+       21504
0-17      15102
Name: Age, dtype: int64
B    231173
C    171175
A    147720
Name: City_Category, dtype: int64
1     193821
2     101838
3      95285
4+     84726
0      74398
Name: Stay_In_Current_City_Years, dtype: int64
M    414259
F    135809
Name: Gender, dtype: int64
4     72308
0     69638
7     59133
1     47426
17    40043
20    33562
12    31179
14    27309
2     26588
16    25371
6     20355
3     17650
10    12930
5     12177
15    12165
11    11586
19     8461
13     7728
18     6622
9      6291
8      1546
Name: Occupation, dtype: int64
0    324731
1    225337
Name: Marital_Status, dtype: int64
5     150933
1     140378
8     113925
11     24287
2      23864
6      20466
3      20213
4      11753
16      9828
15      6290
13      5549
10      5125
12      3947
7       3721
18      3125
20      2550
19      1603
14      1523
17       578
9        410
Name: Product_Category_1, dty

In [12]:
df.isnull().sum()

User_ID                       0
Product_ID                    0
Gender                        0
Age                           0
Occupation                    0
City_Category                 0
Stay_In_Current_City_Years    0
Marital_Status                0
Product_Category_1            0
Product_Category_2            0
Product_Category_3            0
Purchase                      0
dtype: int64

In [9]:
b=['Product_Category_3','Product_Category_2']

In [10]:
for i in b:
    most_common_value=df[i].mode().iloc[0]
    df[i]=df[i].fillna(most_common_value)


In [13]:
X=df.drop(['Purchase'],axis=1)

In [16]:
LE=LabelEncoder()

In [17]:
# it'll convert just strings into categorical .
X=X.apply(LE.fit_transform) # here this apply label encoding on each column independently while using X=LE>fit_transform(X) itll apply label encoding on complete data ye sab ko ak he type ka data samjh kar label encoding apply kardege.

In [18]:
X

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,0,672,0,0,10,0,2,0,2,6,12
1,0,2376,0,0,10,0,2,0,0,4,10
2,0,852,0,0,10,0,2,0,11,6,12
3,0,828,0,0,10,0,2,0,11,12,12
4,1,2734,1,6,16,2,4,0,7,6,12
...,...,...,...,...,...,...,...,...,...,...,...
550063,5883,3567,1,5,13,1,1,1,19,6,12
550064,5885,3568,0,2,1,2,3,0,19,6,12
550065,5886,3568,0,2,15,1,4,1,19,6,12
550066,5888,3568,0,6,1,2,2,0,19,6,12


In [19]:
# now we converting categorical into numerical.
X.Gender=pd.to_numeric(X.Gender)
X.Age=pd.to_numeric(X.Age)
X.City_Category=pd.to_numeric(X.City_Category)
X.Marital_Status=pd.to_numeric(X.Marital_Status)
X.Occupation=pd.to_numeric(X.Occupation)
X.Product_Category_1=pd.to_numeric(X.Product_Category_1)
X.Product_Category_2=pd.to_numeric(X.Product_Category_2)
X.Product_Category_3=pd.to_numeric(X.Product_Category_3)
X.Stay_In_Current_City_Years=pd.to_numeric(X.Stay_In_Current_City_Years)

In [23]:
Y=df['Purchase']

In [24]:
SS=StandardScaler()

In [25]:
XS=SS.fit_transform(X)

#### Using PCA

In [28]:
pc=PCA(4)

In [33]:
principal_component=pc.fit_transform(X)

In [34]:
pc.explained_variance_ratio_

array([7.35041374e-01, 2.64935995e-01, 1.10061180e-05, 6.21704987e-06])

In [35]:
principaldf=pd.DataFrame(data=principal_component,columns=['component1','component2','component3','component4'])

#### Feature Engineering

In [36]:
kf=KFold(20)

#### Splitting Dataset

In [37]:
for a,b in kf.split(principaldf):
    X_train,X_test=XS[a],XS[b]
    y_train,y_test = Y[a],Y[b]


#### Train on Different Models

In [38]:
lr=LinearRegression()
dtr=DecisionTreeRegressor()
rfr=RandomForestRegressor()
gbr=GradientBoostingRegressor()

In [39]:
fit1=lr.fit(X_train,y_train)

In [40]:
fit2=dtr.fit(X_train,y_train)

In [44]:
print('Decision Tree Regression Accuracy',dtr.score(X_train,y_train))

Decision Tree Regression Accuracy 1.0


In [43]:
print('Liner Regression Accuracy',lr.score(X_train,y_train))

Liner Regression Accuracy 0.11829233894211866


In [ ]:
fit3=rfr.fit(X_train,y_train)

In [45]:
fit4=gbr.fit(X_train,y_train)

In [46]:
print('Gradient Boosting Regressor Accuracy',gbr.score(X_train,y_train))

Gradient Boosting Regressor Accuracy 0.6549517152859553
